# Machine Learning Project

Spotify dataset

## Description des données

In [ ]:
spotify_data <- read.table(file = "data/spotify-extr.txt", header = T, sep = " ")

In [ ]:
head(spotify_data)

In [ ]:
dim(spotify_data)

In [ ]:
sapply(spotify_data, class)

In [ ]:
sum(is.na(spotify_data))

In [ ]:
spotify_data$pop.class <- factor(spotify_data$pop.class, order = T)
spotify_data$key <- factor(spotify_data$key)
spotify_data$mode <- factor(spotify_data$mode)

In [ ]:
levels(spotify_data$pop.class)
levels(spotify_data$key)
levels(spotify_data$mode)

In [ ]:
pop.class <- spotify_data$pop.class
key <- spotify_data$key
popularity <- spotify_data$popularity

In [ ]:
percentage <- prop.table(table(pop.class)) * 100
freqs <- cbind(freq = table(pop.class), percentage = percentage)
freqs

In [ ]:
percentage <- prop.table(table(key)) * 100
freqs <- cbind(freq = table(key), percentage = percentage)
freqs

In [ ]:
summary(spotify_data)

## Analyses uni et multidimensionnelles

In [ ]:
library(vcd)

data_quant <- spotify_data[, -c(8, 11, 15)]

options(repr.plot.width = 15, repr.plot.height = 20)
par(mfrow = c(4, 3))
for (i in 1:12) {
    hist(data_quant[, i], main = paste("Histogram of", colnames(data_quant)[i], sep = " "), 
        breaks = ifelse(i == 2, 100, 20), xlab = colnames(data_quant)[i])
}

In [ ]:
spotify_data$log_duration <- log(spotify_data$duration)
hist(spotify_data$log_duration, main = "Histogram of Log of Duration", breaks = 50, 
    xlab = "Duration")

In [ ]:
spotify_data$tempo_norm <- scale(spotify_data$tempo)
hist(spotify_data$tempo_norm, main = "Histogram of Tempo Normalized", breaks = 50, 
    xlab = "Tempo")

In [ ]:
spotify_data$dance_norm <- scale(spotify_data$danceability)
hist(spotify_data$tempo_norm, main = "Histogram of Danceability Normalized", breaks = 50, 
    xlab = "Danceability")

In [ ]:
spotify_data <- spotify_data[, -c(4, 5, 14)]
head(spotify_data)

In [ ]:
pop_vec <- data.frame(pop.class)
pop_vec <- dplyr::count(pop_vec, pop.class)
pop_vec

In [ ]:
library(ggplot2)

p <- ggplot(pop_vec, aes(x = pop.class, y = n, fill = pop.class, label = n)) + geom_col() + 
    geom_text(nudge_y = 20) + ggtitle("Songs per Popularity Class") + xlab("Popularity Class") + 
    ylab("Song Count") + theme_bw() + theme(legend.position = "none") + theme(plot.title = element_text(hjust = 0.5))
p

In [ ]:
key_vec <- data.frame(key)
key_vec <- dplyr::count(key_vec, key)

p <- ggplot(key_vec, aes(x = key, y = n, fill = key, label = n)) + geom_col() + geom_text(nudge_y = 20) + 
    ggtitle("Songs per Key") + xlab("Key") + ylab("Song Count") + theme_bw() + theme(legend.position = "none") + 
    theme(plot.title = element_text(hjust = 0.5))
p

In [ ]:
library(WVPlots)

ScatterHist(spotify_data, "popularity", "year", title = "Popularity vs. Year")

In [ ]:
ScatterHist(spotify_data, "acousticness", "energy", title = "Acousticness vs. Energy")

In [ ]:
library(RColorBrewer)
library(corrplot)

data_quant <- spotify_data[, -c(6, 9, 12)]
M <- cor(data_quant)
corrplot(M, method = "square", col = brewer.pal(n = 10, name = "RdYlBu"))

In [ ]:
attributes = c("acousticness", "energy", "loudness", "popularity")

acoustic <- spotify_data$acousticness
energy <- spotify_data$energy
loud <- spotify_data$loudness

pairs(~popularity + acoustic + energy + loud, data = spotify_data)

In [ ]:
pairs(~., data = data_quant, col = "lightskyblue")

### ACP

In [ ]:
library(FactoMineR)

data_quant <- data_quant[, -c(2)]
data.pca <- PCA(data_quant, scale.unit = T, graph = F, ncp = 6)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))
barplot(data.pca$eig[, 2], ylab = "Percentage", main = "Proportion of Inertia")
boxplot(data.pca$ind$coord, main = "Coordinates of Individuals")

In [ ]:
library(factoextra)

eig.val <- get_eig(data.pca)
eig.val

In [ ]:
plot(data.pca, choix = "varcor")
plot(data.pca, choix = "ind")

In [ ]:
plot(data.pca, choix = "varcor", axes = c(1, 3))

In [ ]:
# Scree plot - Eigenvalues
fviz_eig(data.pca, addlabels = TRUE)

In [ ]:
barplot(eig.val[, 3], col = rgb(70/255, 130/255, 180/255))
abline(h = 80, col = "red", lty = 2)

In [ ]:
var <- get_pca_var(data.pca)
var

In [ ]:
fviz_pca_var(data.pca, col.var = "cos2", gradient.cols = c("#00AFBB", "#E7B800", 
    "#FC4E07"), repel = TRUE)

In [ ]:
corrplot(var$cos2, is.corr = FALSE)
fviz_cos2(data.pca, choice = "var", axes = 1:2)

## Apprentissage

In [ ]:
set.seed(42)
test.ratio <- 0.2
npop <- nrow(spotify_data)
nvar <- ncol(spotify_data)
ntest <- ceiling(npop * test.ratio)
testi <- sample(1:npop, ntest)
appri <- setdiff(1:npop, testi)

In [ ]:
# construction de l'échantillon d'apprentissage
datappr <- spotify_data[appri, -12]
# construction de l'échantillon test
datestr <- spotify_data[testi, -12]
# vérification
str(datappr)
str(datestr)
# summary(datappr)

In [ ]:
plot.res <- function(x, y, titre = "titre") {
    plot(x, y, col = "blue", xlim = c(0, 250), ylim = c(-100, 100), ylab = "Résidus", 
        xlab = "Valeurs prédites", main = titre, pch = 20)
    # points(x2, y, col='red')
    abline(h = 0, col = "green")
}

In [ ]:
reg.lm <- aov(popularity ~ ., data = datappr)

# Extraction des résidus et des valeurs ajustées de ce modèle
res.lm <- reg.lm$residuals
fit.lm <- reg.lm$fitted.values

# Graphe des résidus.
plot.res(fit.lm, res.lm, "Régression linéaire sans sélection de variables")

In [ ]:
summary(reg.lm)

In [ ]:
str(datappr)

In [ ]:
library(glmnet)

# avec des variables quantitatives seulement
reg.lasso.quanti <- glmnet(y = datappr[, 10],
                           x = as.matrix(datappr[, -c(6, 9)]))
# avec toutes les variables, créer d'abord la matrice d'expériences 
# avec 'model.matrix' (penser à retirer l'intercept du modèle)
x.mat <- model.matrix(popularity ~ . - 1, data = datappr)
reg.lasso <- glmnet(y = datappr$popularity, x = x.mat)
options(repr.plot.width = 12, repr.plot.height = 10)
plot(reg.lasso, xvar = "lambda", label = TRUE)
legend("topright", 
       legend = paste(1:ncol(x.mat), " - ", colnames(x.mat)))

In [ ]:
reg.lasso.cv <- cv.glmnet(y = datappr[, 10], x = x.mat)
plot(reg.lasso.cv)

In [ ]:
# valeur estimée
paste("CV estimate of lambda :", round(reg.lasso.cv$lambda.1se, 3))
# modèle correspondant
coef(reg.lasso.cv, s = "lambda.1se")

In [ ]:
# Extraction des valeurs ajustées et des résidus
fit.lasso <- predict(reg.lasso.cv, s = "lambda.min", newx = x.mat)
res.lasso <- datappr$popularity - fit.lasso
# Graphe des résidus
options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))
plot.res(fit.lm, res.lm, "linéaire")
plot.res(fit.lasso, res.lasso, "linéaire, pénalité L1")